In [ ]:
# load the relevant libraries for our analysis
import os
import pandas as pd
import torch
import spacy
from tqdm import tqdm_notebook as tqdm
from torchtext import data
from torchtext import datasets
import random
import numpy as np
import torchtext
import sys

In [2]:
# specify device type
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

This tutorial serves as an introduction to the usage of torchtext a library designed to preprocess text data so it
can be usedin tamdem with pytorch as well as with other deep learning libraries

For this tutorial we will use the Consumer Complaint Database put together by the Consumer Financial 
Protection Bureau (CBPB). More info about the data.



#### Basic text preprocessing

We will leverage Spacy's modern text preprocessing methods to lemmatize, handle some spelling errors, create a pronoun flag, etc. The first step in our code is to load the large english language model.

In [3]:
# load spacy model
nlp = spacy.load('en_core_web_lg')

Spacy is one of the most powerful NLP libraries available for data processing (a word on why spacy is better?)/ The next step is to define the data paths.

In [ ]:
# load spacy model
#nlp = spacy.load('en_core_web_lg')

In [4]:
# default data dir
basepath = '/media/jlealtru/data_files/github/Tutorials'

DATA_DIR = '/media/jlealtru/data_files/github/Tutorials/datasets/text_analytics/financial'

os.listdir(DATA_DIR)

['Consumer_Complaints.csv', 'savedmodels']

Since the dataset is quite big, we are not hosting on github, you can get it from here:
link    
    

In [ ]:
# read the complains data to have a better idea on how this works
df = pd.read_csv(os.path.join(DATA_DIR, 'Consumer_Complaints.csv'), engine='python')
print(f"The dataframe has {len(df):,} observations.")
df.head()

We are interested in predicting the type of issue faced by the customer. In the dataset we have 18 different
types of products.

In [ ]:
df[['Product','Complaint ID']].groupby('Product').count()

In [ ]:
Despite the fact the data has more than 1 million observations, a coursory exploration reveals that the data 
has multiple missing values. Checking the actual distribution of answers.

In [ ]:
df[['Consumer consent provided?','Complaint ID']].groupby('Consumer consent provided?').count()

In [ ]:
print(f"We have complete information for {len(df[df['Consumer consent provided?']=='Consent provided']):,} observations")

In [ ]:
#df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), engine='python')

We filter the data to only have observations with consumer narrative, select relevant columns and shuffle 
the data. We use the pandas native sample function that generates a random sample of lenght n, in this 
case it is  equal to the number of the observations on the filtered dataset. We also set the random stated
to a predifined number so we can replicate the results.

In [ ]:
df=df[df['Consumer consent provided?']=='Consent provided']
df=df[['Complaint ID','Consumer complaint narrative','Product']]
df=df.sample(n=df.shape[0], random_state=10)

In [ ]:
df.head()

In [ ]:
# check the distribution of cases for product
df[['Complaint ID', 'Product']].groupby('Product').count()

In [ ]:
# We see that we have twp classes that are very unbalanced, Virtual Currency and Other financial Services. 
# To improve the performance of the model we merge both into a single class
df['Product'][(df['Product']=='Virtual currency') | (df['Product']=='Other financial service')]='Other financial services'

In [ ]:
# check if this works
df[['Complaint ID', 'Product']].groupby('Product').count()

We reset the index and rename the text containing the text field of our data, we will use
that text in the torchtext context.

In [ ]:
df = df.reset_index(drop=True)
df.head()

In [ ]:
df = df.rename(columns={'Consumer complaint narrative': 'text'})
df.head()

In [ ]:
dict_values_target={k:v for k,v in zip(range(len(df.Product.unique())), df.Product.unique())}
product_lookup=pd.DataFrame([[key,value] for key,value in dict_values_target.items()],
     columns=["product_id","product_text"])
product_lookup.head()

In [ ]:
print(len(df))
df=pd.merge(df,product_lookup, how='left', left_on='Product',right_on='product_text')
print(len(df))

In [ ]:
df=df[['text', 'product_id']]

In [ ]:
Next we create, validation, training and test datasets.

In [ ]:
# create test, train and validation datasets 
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test_= df[~msk]
msk1=np.random.rand(len(test_)) <= 0.5
test=test_[msk1]
val=test_[~msk1]
del test_
#df.iloc[0:700000].to_csv(os.path.join(basepath, 'train.csv'), index=False)
#df.iloc[700000:800000].to_csv(os.path.join(basepath, 'test.csv'), index=False)#
#df.iloc[800000:900000].to_csv(os.path.join(basepath, 'valid.csv'), index=False)

In [ ]:
print(f"The len of train dataset is {len(train):,}, the len of test is {len(test):,} and the len of valid is "+
     f"{len(val):,}")

In [ ]:
#train
#Complaint ID Product
train.to_csv(os.path.join(basepath, 'train.csv'),index=False)
test.to_csv(os.path.join(basepath, 'test.csv'),index=False)
val.to_csv(os.path.join(basepath, 'val.csv'),index=False)

In [ ]:
a=pd.read_csv(os.path.join(basepath, 'test.csv'))
a.columns

In [5]:
# do the tokenizer
tokenize_count = 0

# use custom tokenizer with large spacy model
def tokenizer(text): # create a tokenizer function
    global tokenize_count
    if tokenize_count % 1000 == 0:
        sys.stdout.write('\rDoc: {}'.format(tokenize_count))
        sys.stdout.flush()
    tokenize_count += 1
    return [tok.text for tok in nlp.tokenizer(text)]

In [6]:
TEXT = data.Field(sequential=True, tokenize=tokenizer)
#LABEL = data.Field(sequential=False, 
#                   use_vocab=False, 
#                   pad_token=None, 
#                   unk_token=None)
#LABEL=data.LabelField(dtype=torch.float)
#LABEL = data.LabelField(dtype=torch.float)
LABEL = data.Field(sequential=False, use_vocab=False)

In [7]:
_datafields = [  ('text', TEXT), ('product_id', LABEL)
               # we won't be needing the id, so we pass in None as the field
               ]

In [8]:
from torchtext.data import TabularDataset
from torchtext import data

# define 
SEED = 1234

#  add a line about cudnn feterministic
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
#torch.backends.cudnn.deterministic = True



In [9]:
# Use the tabular dataset split function to create

trn, vld, tst = TabularDataset.splits(
    path="data", # the root directory where the data lies
    train=os.path.join(basepath, 'train.csv'), 
    validation=os.path.join(basepath, 'val.csv'),
    test=os.path.join(basepath, 'test.csv'), 
    format='csv',
    skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=_datafields)

Doc: 363000

In [ ]:
# make sure the process worked fine
trn.fields.items()

In [10]:
# build the vocabulary using glove vectors of 300 dimensions. To limit the size of the vocabulary we limit the 
# vocabulary to 50,000 in size and a minumun occurence of 5 times
TEXT.build_vocab(trn, vectors='glove.42B.300d', min_freq=5, 
                max_size=50000)
LABEL.build_vocab(trn) 

In [ ]:
# print the number of unique tokens and the len of the label categories
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

In [ ]:
trn[0].product_id

In [ ]:
# check the most frequent words in the vocabulary
print(TEXT.vocab.freqs.most_common(20))

In [ ]:
print(LABEL.vocab.stoi)

In [11]:
# create our batch iterator object for training. This will automatically 
# shift our input text forward t+1 for our target data for the language model 
# to predict the next word in the sequence
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    (trn, tst, vld), 
    batch_size=32,
    #bptt_len=35, # specifying the sequence length for back prop through time
    device=device,
    #repeat=False, 
    sort_key=lambda x: len(x.text))

In [ ]:
# check the data split getting an observation from the training iterable
b=next(iter(train_iter))

In [ ]:
b.product_id.shape

We will use a LSTM RNN, a pretty standard model used to classify sequential data. For a refresher of RNN you can check out the good videos of Deep AI (they tend to be a bit theory heavy). The deep learning for coders is another great resource if you are interested in videos of Fastai Some of the most common tasks in pytorch is the classif(link to discussion on where.

The models from torch are saved on the nn module. We define a class that will hold the model, layers and parameters necessary for our sample. You may notice the use of the super parameter (Jason wanna chime in here).

Discussion of the LSTM model 
  - add discussion of the model
  - add reference to the LSTM model and a link to a couple of tutorials.


In [12]:
# define a class that will hold the model as well as the necessary parameters for it to work
import torch.nn as nn
import torch.nn.functional as F


class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        """
        Parameters of the model:
        vocab_size: size of the vocabulary after creating it using the Glove embeddings with the defined thresholds
        embedding_dim: size of the vocabulary embeddings
        hidden_dim: hidden dimensions of the lstm model
        output_dim: number of classes in our data
        n_layers: number of lstm layers
        bidirectional: dummy to specify if this is a bidirectional model
        dropout: droptout parameter for the dropout model
        
        """
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        # return the hidden state and the cell. We will concatenate the last two hidden vectors
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        # apply dropout before passing to fully connected layer
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        
        #define output
        output=self.fc(hidden)
        
        
        return output

In [13]:
# define the parameters for the model and instatiate the model class
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [14]:
# define the pretrained_embeddings
pretrained_embeddings = TEXT.vocab.vectors

In [15]:
# move the embedding weights to the model and move the model to the gpu
model.embedding.weight.data.copy_(pretrained_embeddings)
model.cuda()

RNN(
  (embedding): Embedding(44419, 300)
  (rnn): LSTM(300, 128, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=18, bias=True)
  (dropout): Dropout(p=0.5)
)

In [ ]:
# define our loss and optimizer
#loss_function = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.1)
#nn.

In [ ]:
#import torch.optim as optim

#optimizer = optim.Adam(model.parameters())

In [16]:
import torch.optim as optim
learning_rate = 1e-3

# define our loss function and the parameters for updating the model.
# TALK ABOUT BETAS
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.7, 0.99))
n_tokens = pretrained_embeddings.size(0)

In [ ]:
#help(nn.CrossEntropyLoss)
#import torch.nn.functional as F

In [ ]:
#loss_function = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
#def accuracy(out, labels):
#    return torch.sum(labels.data == out)/float(labels.size)

In [ ]:
torch.cuda.empty_cache()

In [17]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    running_loss = 0.0
    model.train()
    
    for batch_i, batch in enumerate(iterator):
        
        optimizer.zero_grad()
        
        batch.text = batch.text.cuda()
        batch.product_id=batch.product_id.cuda()
        
        predictions = model(batch.text)
        
        #criterion = nn.CrossEntropyLoss()
        #loss = criterion(predictions, batch.product_id)
        #if predictions[0].shape==batch.product_id.shape[0]:
            
    
        loss=criterion(predictions, batch.product_id)
        
        
        #acc = binary_accuracy(predictions, batch.product_id)
        epoch_loss += loss.item()
        running_loss += loss.item()
        correct = (torch.max(predictions, 1)[1] == batch.product_id).sum()
        #print(float(correct)/32)
        if batch_i % 10 == 9:    # print every 10 batches
            print('Epoch: {}, Batch: {}, Avg. Loss: {}, correct{}'.format(epoch + 1,
                                                                          batch_i+1, running_loss/1000,
                                                                          float(correct)/32))        
        loss.backward()
        
        optimizer.step()
        
        #print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(train_iter)))
        #epoch_loss += loss.item()
        
        #correct += (predicted.type(torch.DoubleTensor) == labels).sum()
        #pred=predictions[0]
        #print(torch.max(predictions, 0)[1],batch.product_id)
        #print(torch.max(predictions, 1)[1].shape)
        #correct = (rounded_preds == y).float()
        
        #correct = (output == batch.product_id).float().sum()
        #correct/batch.product_id.shape[0]
        #epoch_acc += acc.item()
        
    return epoch_loss,predictions

In [18]:
# define test function to measure accuracy

def test_data():
    
    # iterate through the test dataset
    for i, batch in enumerate(test_iter):
        
        # move inputs to gpu
        batch.text = batch.text.cuda()
        batch.product_id=batch.product_id.cuda()
        
        predictions = model(batch.text)
        
        loss=criterion(predictions, batch.product_id)
                
        #acc = binary_accuracy(predictions, batch.product_id)
        epoch_loss += loss.item()
        running_loss += loss.item()
        correct = (torch.max(predictions, 1)[1] == batch.product_id).sum()
        #print(float(correct)/32)
        if batch_i % 10 == 9:    # print every 10 batches
            print('Epoch: {}, Batch: {}, Avg. Loss: {}, correct{}'.format(epoch + 1,
                                                                          batch_i+1, running_loss/1000,
                                                                          float(correct)/32))        
        loss.backward()
        
        optimizer.step()
        
        
        

In [19]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    
    #train_loss, train_acc = 
    train(model, train_iter, optimizer, criterion)

Epoch: 1, Batch: 10, Avg. Loss: 0.026571139097213745, correct0.21875
Epoch: 1, Batch: 20, Avg. Loss: 0.04975309860706329, correct0.3125
Epoch: 1, Batch: 30, Avg. Loss: 0.07272353875637054, correct0.25
Epoch: 1, Batch: 40, Avg. Loss: 0.09481553781032562, correct0.28125
Epoch: 1, Batch: 50, Avg. Loss: 0.11656966435909272, correct0.15625
Epoch: 1, Batch: 60, Avg. Loss: 0.13947393691539764, correct0.15625
Epoch: 1, Batch: 70, Avg. Loss: 0.16159438967704773, correct0.21875
Epoch: 1, Batch: 80, Avg. Loss: 0.18271789264678956, correct0.28125
Epoch: 1, Batch: 90, Avg. Loss: 0.204649778008461, correct0.1875
Epoch: 1, Batch: 100, Avg. Loss: 0.22611126458644867, correct0.25
Epoch: 1, Batch: 110, Avg. Loss: 0.24631500613689422, correct0.21875
Epoch: 1, Batch: 120, Avg. Loss: 0.26592341232299804, correct0.34375
Epoch: 1, Batch: 130, Avg. Loss: 0.2854790133237839, correct0.28125
Epoch: 1, Batch: 140, Avg. Loss: 0.3051925513744354, correct0.21875
Epoch: 1, Batch: 150, Avg. Loss: 0.3246028344631195, c

Epoch: 1, Batch: 1240, Avg. Loss: 1.947353601515293, correct0.6875
Epoch: 1, Batch: 1250, Avg. Loss: 1.9595524193644525, correct0.8125
Epoch: 1, Batch: 1260, Avg. Loss: 1.9726376451849938, correct0.625
Epoch: 1, Batch: 1270, Avg. Loss: 1.984670688688755, correct0.625
Epoch: 1, Batch: 1280, Avg. Loss: 1.9964171834588051, correct0.5625
Epoch: 1, Batch: 1290, Avg. Loss: 2.011104438602924, correct0.625
Epoch: 1, Batch: 1300, Avg. Loss: 2.023327373743057, correct0.4375
Epoch: 1, Batch: 1310, Avg. Loss: 2.0363642052412034, correct0.65625
Epoch: 1, Batch: 1320, Avg. Loss: 2.0495529835820196, correct0.40625
Epoch: 1, Batch: 1330, Avg. Loss: 2.0624444597959517, correct0.625
Epoch: 1, Batch: 1340, Avg. Loss: 2.0740543812513352, correct0.6875
Epoch: 1, Batch: 1350, Avg. Loss: 2.0856560512781144, correct0.59375
Epoch: 1, Batch: 1360, Avg. Loss: 2.0977654243707655, correct0.5625
Epoch: 1, Batch: 1370, Avg. Loss: 2.1104759133458137, correct0.71875
Epoch: 1, Batch: 1380, Avg. Loss: 2.121829558074474,

Epoch: 1, Batch: 2460, Avg. Loss: 3.3454852415323257, correct0.46875
Epoch: 1, Batch: 2470, Avg. Loss: 3.356063643217087, correct0.5625
Epoch: 1, Batch: 2480, Avg. Loss: 3.366893474638462, correct0.75
Epoch: 1, Batch: 2490, Avg. Loss: 3.3764487593770025, correct0.75
Epoch: 1, Batch: 2500, Avg. Loss: 3.3875157591700553, correct0.6875
Epoch: 1, Batch: 2510, Avg. Loss: 3.3987051785588265, correct0.625
Epoch: 1, Batch: 2520, Avg. Loss: 3.4096121926903726, correct0.6875
Epoch: 1, Batch: 2530, Avg. Loss: 3.4204845309853553, correct0.65625
Epoch: 1, Batch: 2540, Avg. Loss: 3.4314557195305824, correct0.65625
Epoch: 1, Batch: 2550, Avg. Loss: 3.441981392621994, correct0.78125
Epoch: 1, Batch: 2560, Avg. Loss: 3.451146767258644, correct0.71875
Epoch: 1, Batch: 2570, Avg. Loss: 3.46082269769907, correct0.75
Epoch: 1, Batch: 2580, Avg. Loss: 3.472741808116436, correct0.5
Epoch: 1, Batch: 2590, Avg. Loss: 3.482730866074562, correct0.71875
Epoch: 1, Batch: 2600, Avg. Loss: 3.4931316244602204, correc

Epoch: 1, Batch: 3680, Avg. Loss: 4.584212680786848, correct0.625
Epoch: 1, Batch: 3690, Avg. Loss: 4.594221419900656, correct0.78125
Epoch: 1, Batch: 3700, Avg. Loss: 4.604053996175527, correct0.625
Epoch: 1, Batch: 3710, Avg. Loss: 4.614258440583944, correct0.6875
Epoch: 1, Batch: 3720, Avg. Loss: 4.624130010396242, correct0.59375
Epoch: 1, Batch: 3730, Avg. Loss: 4.634084545224905, correct0.59375
Epoch: 1, Batch: 3740, Avg. Loss: 4.643448701947928, correct0.59375
Epoch: 1, Batch: 3750, Avg. Loss: 4.653319998770952, correct0.6875
Epoch: 1, Batch: 3760, Avg. Loss: 4.662553838044405, correct0.6875
Epoch: 1, Batch: 3770, Avg. Loss: 4.671973963052034, correct0.78125
Epoch: 1, Batch: 3780, Avg. Loss: 4.681945736378431, correct0.6875
Epoch: 1, Batch: 3790, Avg. Loss: 4.692024988979101, correct0.65625
Epoch: 1, Batch: 3800, Avg. Loss: 4.701226132720709, correct0.71875
Epoch: 1, Batch: 3810, Avg. Loss: 4.710800917714834, correct0.59375
Epoch: 1, Batch: 3820, Avg. Loss: 4.720688087314367, cor

Epoch: 1, Batch: 4910, Avg. Loss: 5.744577768683434, correct0.75
Epoch: 1, Batch: 4920, Avg. Loss: 5.753668184220791, correct0.59375
Epoch: 1, Batch: 4930, Avg. Loss: 5.762391548097134, correct0.6875
Epoch: 1, Batch: 4940, Avg. Loss: 5.771063171744347, correct0.59375
Epoch: 1, Batch: 4950, Avg. Loss: 5.780379305422306, correct0.65625
Epoch: 1, Batch: 4960, Avg. Loss: 5.790175953686237, correct0.71875
Epoch: 1, Batch: 4970, Avg. Loss: 5.798614360570908, correct0.75
Epoch: 1, Batch: 4980, Avg. Loss: 5.806444496691227, correct0.84375
Epoch: 1, Batch: 4990, Avg. Loss: 5.816188475489616, correct0.78125
Epoch: 1, Batch: 5000, Avg. Loss: 5.825494023144246, correct0.5625
Epoch: 1, Batch: 5010, Avg. Loss: 5.835170646607875, correct0.6875
Epoch: 1, Batch: 5020, Avg. Loss: 5.844102110981941, correct0.59375
Epoch: 1, Batch: 5030, Avg. Loss: 5.852106291472912, correct0.40625
Epoch: 1, Batch: 5040, Avg. Loss: 5.862112437903881, correct0.71875
Epoch: 1, Batch: 5050, Avg. Loss: 5.871592355489731, corr

...

Epoch: 5, Batch: 3940, Avg. Loss: 2.2604715231508017, correct0.8125
Epoch: 5, Batch: 3950, Avg. Loss: 2.2662065366655586, correct0.84375
Epoch: 5, Batch: 3960, Avg. Loss: 2.2716323557943108, correct0.78125
Epoch: 5, Batch: 3970, Avg. Loss: 2.2769305697381497, correct0.75
Epoch: 5, Batch: 3980, Avg. Loss: 2.2826181370019913, correct0.6875
Epoch: 5, Batch: 3990, Avg. Loss: 2.2879691972136498, correct0.84375
Epoch: 5, Batch: 4000, Avg. Loss: 2.293416396796703, correct0.875
Epoch: 5, Batch: 4010, Avg. Loss: 2.2986838988363743, correct0.84375
Epoch: 5, Batch: 4020, Avg. Loss: 2.3047164549529553, correct0.84375
Epoch: 5, Batch: 4030, Avg. Loss: 2.3097943409979345, correct0.71875
Epoch: 5, Batch: 4040, Avg. Loss: 2.315167609214783, correct0.8125
Epoch: 5, Batch: 4050, Avg. Loss: 2.3210956726670267, correct0.71875
Epoch: 5, Batch: 4060, Avg. Loss: 2.3279632851183414, correct0.59375
Epoch: 5, Batch: 4070, Avg. Loss: 2.332905189394951, correct0.8125
Epoch: 5, Batch: 4080, Avg. Loss: 2.3376433758

Epoch: 5, Batch: 5160, Avg. Loss: 2.965608790665865, correct0.75
Epoch: 5, Batch: 5170, Avg. Loss: 2.9717572995722294, correct0.8125
Epoch: 5, Batch: 5180, Avg. Loss: 2.9778300819694996, correct0.6875
Epoch: 5, Batch: 5190, Avg. Loss: 2.983962042450905, correct0.78125
Epoch: 5, Batch: 5200, Avg. Loss: 2.9882843667715786, correct0.8125
Epoch: 5, Batch: 5210, Avg. Loss: 2.9933236475735905, correct0.90625
Epoch: 5, Batch: 5220, Avg. Loss: 2.9994555156081915, correct0.71875
Epoch: 5, Batch: 5230, Avg. Loss: 3.0061200162023307, correct0.875
Epoch: 5, Batch: 5240, Avg. Loss: 3.0126497513502835, correct0.78125
Epoch: 5, Batch: 5250, Avg. Loss: 3.0183882923573258, correct0.9375
Epoch: 5, Batch: 5260, Avg. Loss: 3.024723765030503, correct0.71875
Epoch: 5, Batch: 5270, Avg. Loss: 3.031051168605685, correct0.84375
Epoch: 5, Batch: 5280, Avg. Loss: 3.03728697322309, correct0.90625
Epoch: 5, Batch: 5290, Avg. Loss: 3.0431258372813463, correct0.96875
Epoch: 5, Batch: 5300, Avg. Loss: 3.0491301736980

Epoch: 5, Batch: 6380, Avg. Loss: 3.676474677070975, correct0.78125
Epoch: 5, Batch: 6390, Avg. Loss: 3.6816699725836517, correct0.8125
Epoch: 5, Batch: 6400, Avg. Loss: 3.687191658839583, correct0.875
Epoch: 5, Batch: 6410, Avg. Loss: 3.6931628068238496, correct0.90625
Epoch: 5, Batch: 6420, Avg. Loss: 3.6993062820881604, correct0.875
Epoch: 5, Batch: 6430, Avg. Loss: 3.705435231372714, correct0.84375
Epoch: 5, Batch: 6440, Avg. Loss: 3.7111226970106364, correct0.71875
Epoch: 5, Batch: 6450, Avg. Loss: 3.7170478125959634, correct0.75
Epoch: 5, Batch: 6460, Avg. Loss: 3.721653502210975, correct0.8125
Epoch: 5, Batch: 6470, Avg. Loss: 3.726925511106849, correct0.875
Epoch: 5, Batch: 6480, Avg. Loss: 3.732769710510969, correct0.875
Epoch: 5, Batch: 6490, Avg. Loss: 3.7387728589773177, correct0.625
Epoch: 5, Batch: 6500, Avg. Loss: 3.7439216818362473, correct0.8125
Epoch: 5, Batch: 6510, Avg. Loss: 3.7477517598867416, correct0.84375
Epoch: 5, Batch: 6520, Avg. Loss: 3.7536874345541, corre

Epoch: 5, Batch: 7600, Avg. Loss: 4.385554033339024, correct0.8125
Epoch: 5, Batch: 7610, Avg. Loss: 4.392164447724819, correct0.78125
Epoch: 5, Batch: 7620, Avg. Loss: 4.399462691187859, correct0.75
Epoch: 5, Batch: 7630, Avg. Loss: 4.404812942802906, correct0.8125
Epoch: 5, Batch: 7640, Avg. Loss: 4.4103417261838915, correct0.59375
Epoch: 5, Batch: 7650, Avg. Loss: 4.415692191272974, correct0.90625
Epoch: 5, Batch: 7660, Avg. Loss: 4.421420539826155, correct0.78125
Epoch: 5, Batch: 7670, Avg. Loss: 4.427254356265068, correct0.78125
Epoch: 5, Batch: 7680, Avg. Loss: 4.431578990250826, correct0.8125
Epoch: 5, Batch: 7690, Avg. Loss: 4.436879957795143, correct0.75
Epoch: 5, Batch: 7700, Avg. Loss: 4.443186437696219, correct0.78125
Epoch: 5, Batch: 7710, Avg. Loss: 4.448469536811113, correct0.84375
Epoch: 5, Batch: 7720, Avg. Loss: 4.453950384378433, correct0.84375
Epoch: 5, Batch: 7730, Avg. Loss: 4.459826271027326, correct0.875
Epoch: 5, Batch: 7740, Avg. Loss: 4.4661513015329835, corr

Epoch: 5, Batch: 8820, Avg. Loss: 5.103987958535552, correct0.8125
Epoch: 5, Batch: 8830, Avg. Loss: 5.1096216872483495, correct0.90625
Epoch: 5, Batch: 8840, Avg. Loss: 5.1159859045296905, correct0.9375
Epoch: 5, Batch: 8850, Avg. Loss: 5.1220312188714745, correct0.8125
Epoch: 5, Batch: 8860, Avg. Loss: 5.128596181973815, correct0.8125
Epoch: 5, Batch: 8870, Avg. Loss: 5.134440663680434, correct0.65625
Epoch: 5, Batch: 8880, Avg. Loss: 5.139386487051844, correct0.8125
Epoch: 5, Batch: 8890, Avg. Loss: 5.145646063223481, correct0.9375
Epoch: 5, Batch: 8900, Avg. Loss: 5.152067774936556, correct0.84375
Epoch: 5, Batch: 8910, Avg. Loss: 5.157663907900453, correct0.875
Epoch: 5, Batch: 8920, Avg. Loss: 5.163537050947547, correct0.6875
Epoch: 5, Batch: 8930, Avg. Loss: 5.1691228403002025, correct0.8125
Epoch: 5, Batch: 8940, Avg. Loss: 5.175497365444898, correct0.84375
Epoch: 5, Batch: 8950, Avg. Loss: 5.181607965976, correct0.875
Epoch: 5, Batch: 8960, Avg. Loss: 5.188133324176073, correc

In [20]:
# save model
## TODO: change the name to something uniqe for each new model
model_dir = os.path.join(DATA_DIR, 'savedmodels/')
model_name = 'simple_lstm_financial_data.pt'

# after training, save your model parameters in the dir 'saved_models'
torch.save(model.state_dict(), model_dir+model_name)

In [ ]:
#torch.nn.functional.softmax(pr1[0,:])
print(b1)
print(torch.max(torch.nn.functional.softmax(pr1), 1)[1])
#(np.log(pr1[0,:]))

In [ ]:
train

In [ ]:
# save model once its done.

In [ ]:
model